<a href="https://colab.research.google.com/github/spetrescu/delimiter-recongition-tokenizer/blob/main/character_level_named_entity_recognition_rnns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

print(tf.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.9.2


In [ ]:
# get code and data
!git clone https://github.com/spetrescu/delimiter-recongition-tokenizer.git

Cloning into 'delimiter-recongition-tokenizer'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [ ]:
cd delimiter-recongition-tokenizer

/content/delimiter-recongition-tokenizer


In [ ]:
!ls

README.md  synthetic_logs_10k.csv


In [ ]:
# read data
synthetic_logs = []
synthetic_logs_labels = []

a = 0
b = 0
train_examples = []
test_examples = []
with open("synthetic_logs_10k.csv") as f:
    next(f)
    for line in f:
      a += 1
      if a <= 2559:
        csv_row = line.split(",")
        synthetic_logs.append(list(csv_row[0]))
        synthetic_logs_labels.append(list(str(csv_row[1]).strip()))
        train_examples.append([[list(csv_row[0]), list(str(csv_row[1]).strip())]])
      else:
        b += 1
        if b <= 255:
          csv_row = line.split(",")
          test_examples.append([[list(csv_row[0]), list(str(csv_row[1]).strip())]])
        else:
          break
      

for el in train_examples:
  print(el[0][0])
  print(el[0][1])
  break

print("hey")
for el in test_examples:
  print(el[0][0])
  print(el[0][1])
  break

['s', 'w', 'a', 'y', ' ', 'y', 'e', 's', 'e', 's', ' ', 't', 'h', 'a', 't', ' ', 'a', 'm', ':', 'f', 'i', 'v', 'e', ':', 'L', 'o', 'g', ' ', 't', 'r', 'y', ' ', 'j', 'a', 'y', ' ', 'g', 'i', 'v', 'e', 'n', ' ', '9', '0', ' ', 's', 'j', 'd', 'f', 's', '8', '3', '4']
['O', 'O', 'O', 'O', 'D', 'O', 'O', 'O', 'O', 'O', 'D', 'O', 'O', 'O', 'O', 'D', 'O', 'O', 'D', 'O', 'O', 'O', 'O', 'D', 'O', 'O', 'O', 'D', 'O', 'O', 'O', 'D', 'O', 'O', 'O', 'D', 'O', 'O', 'O', 'O', 'O', 'D', 'O', 'O', 'D', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
hey
['j', 'a', 'y', ' ', 's', 'w', 'a', 'y', ' ', 'h', 'i', 'd', 'e', ' ', 'j', 'a', 'y', ' ', 't', 'r', 'y', ' ', 'a', 'm', ' ', 'h', 'i', 'd', 'e', ' ', 'j', 'a', 'y', ' ', '3', '0', ';', 'm', 'i', 'g', 'h', 't', ' ', '6', '4', '5', '3']
['O', 'O', 'O', 'D', 'O', 'O', 'O', 'O', 'D', 'O', 'O', 'O', 'O', 'D', 'O', 'O', 'O', 'D', 'O', 'O', 'O', 'D', 'O', 'O', 'D', 'O', 'O', 'O', 'O', 'D', 'O', 'O', 'O', 'D', 'O', 'O', 'D', 'O', 'O', 'O', 'O', 'O', 'D', 'O', 'O', 'O

In [ ]:
all_text = " ".join([" ".join(x[0][0]) for x in train_examples+test_examples])
# print(all_text)
vocab = sorted(set(all_text))
# print(vocab)

all_labels = "".join(["".join(x[0][1]) for x in train_examples+test_examples])
labels = sorted(set(all_labels))
    
# create character/id and label/id mapping
char2idx = {u:i+1 for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
label2idx = {u:i+1 for i, u in enumerate(labels)}
idx2label = np.array(labels)
print(idx2label)
print(char2idx)

['D' 'O']
{' ': 1, '0': 2, '1': 3, '2': 4, '3': 5, '4': 6, '5': 7, '6': 8, '8': 9, '9': 10, ':': 11, ';': 12, 'B': 13, 'D': 14, 'H': 15, 'J': 16, 'L': 17, 'U': 18, 'X': 19, 'a': 20, 'b': 21, 'c': 22, 'd': 23, 'e': 24, 'f': 25, 'g': 26, 'h': 27, 'i': 28, 'j': 29, 'k': 30, 'l': 31, 'm': 32, 'n': 33, 'o': 34, 'r': 35, 's': 36, 't': 37, 'u': 38, 'v': 39, 'w': 40, 'x': 41, 'y': 42}


In [ ]:
train_formatted = []
test_formatted = []

for el in train_examples:
  train_formatted.append([[char2idx[x] for x in el[0][0]],np.array([label2idx[x] for x in el[0][1]])])

for el in test_examples:
  test_formatted.append([[char2idx[x] for x in el[0][0]],np.array([label2idx[x] for x in el[0][1]])])

for el in train_formatted:
  print(el)
  break

for el in test_formatted:
  print(el)
  break

[[36, 40, 20, 42, 1, 42, 24, 36, 24, 36, 1, 37, 27, 20, 37, 1, 20, 32, 11, 25, 28, 39, 24, 11, 17, 34, 26, 1, 37, 35, 42, 1, 29, 20, 42, 1, 26, 28, 39, 24, 33, 1, 10, 2, 1, 36, 29, 23, 25, 36, 9, 5, 6], array([2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2,
       2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2,
       1, 2, 2, 2, 2, 2, 2, 2, 2])]
[[29, 20, 42, 1, 36, 40, 20, 42, 1, 27, 28, 23, 24, 1, 29, 20, 42, 1, 37, 35, 42, 1, 20, 32, 1, 27, 28, 23, 24, 1, 29, 20, 42, 1, 5, 2, 12, 32, 28, 26, 27, 37, 1, 8, 6, 7, 5], array([2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1,
       2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2])]


In [ ]:
valid_formatted = train_formatted.copy()

for eg in train_formatted:
  print(eg[0])
  print(eg[1])
  break

# training generator
def gen_train_series():
    for eg in train_formatted:
      yield eg[0],eg[1]

# validation generator
def gen_valid_series():
    for eg in valid_formatted:
      yield eg[0],eg[1]

# test generator
def gen_test_series():
  for eg in test_formatted:
      yield eg[0],eg[1]
  
# create Dataset objects for train, test and validation sets  
series = tf.data.Dataset.from_generator(gen_train_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))
series_valid = tf.data.Dataset.from_generator(gen_valid_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))
series_test = tf.data.Dataset.from_generator(gen_test_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))

BATCH_SIZE = 1
BUFFER_SIZE=1000

# create padded batch series objects for train, test and validation sets
ds_series_batch = series.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)
ds_series_batch_valid = series_valid.padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)
ds_series_batch_test = series_test.padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)

# print example batches
for input_example_batch, target_example_batch in ds_series_batch.take(1):
  print(input_example_batch)
  print(target_example_batch)

[36, 40, 20, 42, 1, 42, 24, 36, 24, 36, 1, 37, 27, 20, 37, 1, 20, 32, 11, 25, 28, 39, 24, 11, 17, 34, 26, 1, 37, 35, 42, 1, 29, 20, 42, 1, 26, 28, 39, 24, 33, 1, 10, 2, 1, 36, 29, 23, 25, 36, 9, 5, 6]
[2 2 2 2 1 2 2 2 2 2 1 2 2 2 2 1 2 2 1 2 2 2 2 1 2 2 2 1 2 2 2 1 2 2 2 1 2
 2 2 2 2 1 2 2 1 2 2 2 2 2 2 2 2]
tf.Tensor(
[[25 28 39 24 12 37 35 38 24  1 42 24 36  1 33 34 36  1  4  5  6 10  1 22
  31 20 42  1 27 28 23 24  1 20 32  1  3  4  5  6  1 26 28 39 24 33  1 28
  36]], shape=(1, 49), dtype=int32)
tf.Tensor(
[[2 2 2 2 1 2 2 2 2 1 2 2 2 1 2 2 2 1 2 2 2 2 1 2 2 2 2 1 2 2 2 2 1 2 2 1
  2 2 2 2 1 2 2 2 2 2 1 2 2]], shape=(1, 49), dtype=int32)


In [ ]:
vocab_size = len(vocab)+1

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

label_size = len(labels)  

# build LSTM model
def build_model(vocab_size,label_size, embedding_dim, rnn_units, batch_size):
      model = tf.keras.Sequential([
          tf.keras.layers.Embedding(vocab_size, embedding_dim,
                            batch_input_shape=[batch_size, None],mask_zero=True),
          tf.keras.layers.LSTM(rnn_units,
                      return_sequences=True,
                      stateful=True,
                      recurrent_initializer='glorot_uniform'),
          tf.keras.layers.Dense(label_size)
          ])
      return model

model = build_model(
      vocab_size = len(vocab)+1,
      label_size=len(labels)+1,
      embedding_dim=embedding_dim,
      rnn_units=rnn_units,
      batch_size=BATCH_SIZE)

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (1, None, 256)            11008     
                                                                 
 lstm_2 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 dense_2 (Dense)             (1, None, 3)              3075      
                                                                 
Total params: 5,261,059
Trainable params: 5,261,059
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import os

# define loss function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss,metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS=2

history = model.fit(ds_series_batch, epochs=EPOCHS, validation_data=ds_series_batch_valid,callbacks=[checkpoint_callback])

Epoch 1/2
2559/2559 [==============================] - 68s 25ms/step - loss: 0.0065 - sparse_categorical_accuracy: 0.9973 - val_loss: 1.2535e-06 - val_sparse_categorical_accuracy: 1.0000
Epoch 2/2
2559/2559 [==============================] - 64s 25ms/step - loss: 5.6644e-07 - sparse_categorical_accuracy: 1.0000 - val_loss: 2.2188e-07 - val_sparse_categorical_accuracy: 1.0000


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

preds = np.array([])
y_trues= np.array([])

# iterate through test set, make predictions based on trained model
for input_example_batch, target_example_batch in ds_series_batch_test:
  pred=model.predict(input_example_batch, batch_size=1)
  pred_max=tf.argmax(tf.nn.softmax(pred),2).numpy().flatten()
  y_true=target_example_batch.numpy().flatten()
  print(input_example_batch)
  print(pred_max)
  preds=np.concatenate([preds,pred_max])
  y_trues=np.concatenate([y_trues,y_true])
  print(y_trues)
  break

# remove padding from evaluation
remove_padding = [(p,y) for p,y in zip(preds,y_trues) if y!=0]

r_p = [x[0] for x in remove_padding]
r_t = [x[1] for x in remove_padding]

# print confusion matrix and classification report
print(confusion_matrix(r_p,r_t))
print(classification_report(r_p,r_t))


1/1 [==============================] - 0s 20ms/step
tf.Tensor(
[[29 20 42  1 36 40 20 42  1 27 28 23 24  1 29 20 42  1 37 35 42  1 20 32
   1 27 28 23 24  1 29 20 42  1  5  2 12 32 28 26 27 37  1  8  6  7  5]], shape=(1, 47), dtype=int32)
[2 2 2 1 2 2 2 2 1 2 2 2 2 1 2 2 2 1 2 2 2 1 2 2 1 2 2 2 2 1 2 2 2 1 2 2 1
 2 2 2 2 2 1 2 2 2 2]
[2. 2. 2. 1. 2. 2. 2. 2. 1. 2. 2. 2. 2. 1. 2. 2. 2. 1. 2. 2. 2. 1. 2. 2.
 1. 2. 2. 2. 2. 1. 2. 2. 2. 1. 2. 2. 1. 2. 2. 2. 2. 2. 1. 2. 2. 2. 2.]
[[10  0]
 [ 0 37]]
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00        10
         2.0       1.00      1.00      1.00        37

    accuracy                           1.00        47
   macro avg       1.00      1.00      1.00        47
weighted avg       1.00      1.00      1.00        47



In [ ]:
a = 0

for input_example_batch, target_example_batch in ds_series_batch_test:
  pred = model.predict(input_example_batch, batch_size=1)
  pred_max=tf.argmax(tf.nn.softmax(pred),2).numpy().flatten()
  # p = np.argmax(p, axis=-1)
  print("{:14} ({:5}): {}".format("Word", "True", "Pred"))
  print(input_example_batch)
  print(pred_max)
  # for w,pred in zip(input_example_batch,p[0]):
  #   print("{:14}: {}".format(words[w],tags[pred]))
  # print()

  print([idx2char[x-1] for x in input_example_batch.numpy()[0]])
  print([idx2label[x-1] for x in target_example_batch.numpy()[0]])
  print([idx2label[x-1] for x in pred_max])
  # print([,np.array([label2idx[x] for x in pred_max.numpy()[0]])])
  a += 1
  if a == 10:
    break
# for i in range(10):
#   print(X_test[i])
#   p = model.predict(np.array([X_test[i]]))
#   p = np.argmax(p, axis=-1)
#   print("{:14} ({:5}): {}".format("Word", "True", "Pred"))
#   for w,pred in zip(X_test[i],p[0]):
#       print("{:14}: {}".format(words[w],tags[pred]))
#   print()

1/1 [==============================] - 0s 18ms/step
Word           (True ): Pred
tf.Tensor(
[[29 20 42  1 36 40 20 42  1 27 28 23 24  1 29 20 42  1 37 35 42  1 20 32
   1 27 28 23 24  1 29 20 42  1  5  2 12 32 28 26 27 37  1  8  6  7  5]], shape=(1, 47), dtype=int32)
[2 2 2 1 2 2 2 2 1 2 2 2 2 1 2 2 2 1 2 2 2 1 2 2 1 2 2 2 2 1 2 2 2 1 2 2 1
 2 2 2 2 2 1 2 2 2 2]
['j', 'a', 'y', ' ', 's', 'w', 'a', 'y', ' ', 'h', 'i', 'd', 'e', ' ', 'j', 'a', 'y', ' ', 't', 'r', 'y', ' ', 'a', 'm', ' ', 'h', 'i', 'd', 'e', ' ', 'j', 'a', 'y', ' ', '3', '0', ';', 'm', 'i', 'g', 'h', 't', ' ', '6', '4', '5', '3']
['O', 'O', 'O', 'D', 'O', 'O', 'O', 'O', 'D', 'O', 'O', 'O', 'O', 'D', 'O', 'O', 'O', 'D', 'O', 'O', 'O', 'D', 'O', 'O', 'D', 'O', 'O', 'O', 'O', 'D', 'O', 'O', 'O', 'D', 'O', 'O', 'D', 'O', 'O', 'O', 'O', 'O', 'D', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'D', 'O', 'O', 'O', 'O', 'D', 'O', 'O', 'O', 'O', 'D', 'O', 'O', 'O', 'D', 'O', 'O', 'O', 'D', 'O', 'O', 'D', 'O', 'O', 'O', 'O', 'D', 'O', 'O', 'O

In [ ]:
# training generator

train_for = []

train_for.append([[29, 20, 42, 1, 36, 40, 20, 42, 1, 27, 28, 23, 24, 1, 29, 20, 42, 1, 37, 35, 42, 1, 20, 32, 1, 27, 28, 23, 24, 1, 29, 20, 42, 1, 5, 2, 12, 32, 28, 26, 27, 37, 1, 8, 6, 7, 5,29, 20, 42, 1, 36, 40, 20, 42, 1, 27, 28, 23, 24, 1, 29, 20, 42, 1, 37, 35, 42, 1, 20, 32, 1, 27, 28, 23, 24, 1, 29, 20, 42, 1, 5, 2, 12, 32, 28, 26, 27, 37, 1, 8, 6, 7, 5,29, 20, 42, 1, 36, 40, 20, 42, 1, 27, 28, 23, 24, 1, 29, 20, 42, 1, 37, 35, 42, 1, 20, 32, 1, 27, 28, 23, 24, 1, 29, 20, 42, 1, 5, 2, 12, 32, 28, 26, 27, 37, 1, 8, 6, 7, 5,29, 20, 42, 1, 36, 40, 20, 42, 1, 27, 28, 23, 24, 1, 29, 20, 42, 1, 37, 35, 42, 1, 20, 32, 1, 27, 28, 23, 24, 1, 29, 20, 42, 1, 5, 2, 12, 32, 28, 26, 27, 37, 1, 8, 6, 7, 5,29, 20, 42, 1, 36, 40, 20, 42, 1, 27, 28, 23, 24, 1, 29, 20, 42, 1, 37, 35, 42, 1, 20, 32, 1, 27, 28, 23, 24, 1, 29, 20, 42, 1, 5, 2, 12, 32, 28, 26, 27, 37, 1, 8, 6, 7, 5,29, 20, 42, 1, 36, 40, 20, 42, 1, 27, 28, 23, 24, 1, 29, 20, 42, 1, 37, 35, 42, 1, 20, 32, 1, 27, 28, 23, 24, 1, 29, 20, 42, 1, 5, 2, 12, 32, 28, 26, 27, 37, 1, 8, 6, 7, 5,29, 20, 42, 1, 36, 40, 20, 42, 1, 27, 28, 23, 24, 1, 29, 20, 42, 1, 37, 35, 42, 1, 20, 32, 1, 27, 28, 23, 24, 1, 29, 20, 42, 1, 5, 2, 12, 32, 28, 26, 27, 37, 1, 8, 6, 7, 5,29, 20, 42, 1, 36, 40, 20, 42, 1, 27, 28, 23, 24, 1, 29, 20, 42, 1, 37, 35, 42, 1, 20, 32, 1, 27, 28, 23, 24, 1, 29, 20, 42, 1, 5, 2, 12, 32, 28, 26, 27, 37, 1, 8, 6, 7, 5,29, 20, 42, 1, 36, 40, 20, 42, 1, 27, 28, 23, 24, 1, 29, 20, 42, 1, 37, 35, 42, 1, 20, 32, 1, 27, 28, 23, 24, 1, 29, 20, 42, 1, 5, 2, 12, 32, 28, 26, 27, 37, 1, 8, 6, 7, 5], np.array([2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1,
       2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2,2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1,
       2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2,2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1,
       2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2,2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1,
       2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2,2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1,
       2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2,2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1,
       2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2,2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1,
       2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2,2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1,
       2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2,2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1,
       2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2])])

def gen_train_series_2():
    for eg in train_for:
      yield eg[0],eg[1]

series_test_2 = tf.data.Dataset.from_generator(gen_train_series_2,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))
ds_series_batch_test_2 = series_test_2.padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)

In [ ]:
a = 0

for input_example_batch, target_example_batch in ds_series_batch_test_2:
  pred = model.predict(input_example_batch, batch_size=1)
  pred_max=tf.argmax(tf.nn.softmax(pred),2).numpy().flatten()
  # p = np.argmax(p, axis=-1)
  print("{:14} ({:5}): {}".format("Word", "True", "Pred"))
  print(input_example_batch)
  print(pred_max)
  # for w,pred in zip(input_example_batch,p[0]):
  #   print("{:14}: {}".format(words[w],tags[pred]))
  # print()

  print([idx2char[x-1] for x in input_example_batch.numpy()[0]])
  print([idx2label[x-1] for x in target_example_batch.numpy()[0]])
  print([idx2label[x-1] for x in pred_max])

  print("Log message test: \n" + "".join([idx2char[x-1] for x in input_example_batch.numpy()[0]]))
  print("".join([idx2label[x-1] for x in pred_max])+ "\nResults")

  # print([,np.array([label2idx[x] for x in pred_max.numpy()[0]])])
  break

1/1 [==============================] - 0s 72ms/step
Word           (True ): Pred
tf.Tensor(
[[29 20 42  1 36 40 20 42  1 27 28 23 24  1 29 20 42  1 37 35 42  1 20 32
   1 27 28 23 24  1 29 20 42  1  5  2 12 32 28 26 27 37  1  8  6  7  5 29
  20 42  1 36 40 20 42  1 27 28 23 24  1 29 20 42  1 37 35 42  1 20 32  1
  27 28 23 24  1 29 20 42  1  5  2 12 32 28 26 27 37  1  8  6  7  5 29 20
  42  1 36 40 20 42  1 27 28 23 24  1 29 20 42  1 37 35 42  1 20 32  1 27
  28 23 24  1 29 20 42  1  5  2 12 32 28 26 27 37  1  8  6  7  5 29 20 42
   1 36 40 20 42  1 27 28 23 24  1 29 20 42  1 37 35 42  1 20 32  1 27 28
  23 24  1 29 20 42  1  5  2 12 32 28 26 27 37  1  8  6  7  5 29 20 42  1
  36 40 20 42  1 27 28 23 24  1 29 20 42  1 37 35 42  1 20 32  1 27 28 23
  24  1 29 20 42  1  5  2 12 32 28 26 27 37  1  8  6  7  5 29 20 42  1 36
  40 20 42  1 27 28 23 24  1 29 20 42  1 37 35 42  1 20 32  1 27 28 23 24
   1 29 20 42  1  5  2 12 32 28 26 27 37  1  8  6  7  5 29 20 42  1 36 40
  20 42  1 27 28 23 